<td>
<a href="https://colab.research.google.com/github/raoulg/straattaal/blob/main/notebooks/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</td>

In [ ]:
try:
   import google.colab
   IN_COLAB = True
except ImportError:
   IN_COLAB = False

if IN_COLAB:
   !pip install slanggen

In [1]:
from pathlib import Path
from slanggen import datatools
from slanggen import models
import torch

I scraped the "Amsterdamse straattaal woordenboek" , see [link](https://www.mijnwoordenboek.nl/dialect/Amsterdamse%20straattaal)

In [ ]:
processed_words = datatools.load_data(Path("../assets/straattaal.txt"))
processed_words[:10], len(processed_words)

We have 453 words in total. I added a start `<s>` and stop `</s>` tag.
We will use a Byte Pair Encoding (BPE) algorithm to learn the subword units from the corpus.

In [ ]:
tokenizer = models.buildBPE(corpus=processed_words, vocab_size=100)
list(tokenizer.get_vocab())[:10]

We can now encode a word and see which tokens are created:

In [ ]:
enc = tokenizer.encode("waggie")
enc.tokens

And reconstruct the word

In [ ]:
tokenizer.decode(enc.ids)

Let's process the sequences. We will:
- transform words into subtokens, and then into arbitrary integers
- add zeros to make all sequences the same length (padding)

In [ ]:
padded_sequences = datatools.preprocess(processed_words, tokenizer)
padded_sequences

Every word now is a list of integers. We will shift the sequence one position, such that the target (to predict) is the next token in the sequence.

In [ ]:
dataset = datatools.ShiftedDataset(padded_sequences)
dataset

In [ ]:
x, y = dataset[0]
print(f"input: {x}")
print(f"output: {y}")

we will use a Dataloader. This will batch the sequences and shuffle the dataset.

In [ ]:
# import torch dataloader
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=16, shuffle=True)
x, y = next(iter(loader))
x.shape, y.shape

Lets look at the full dataset:

In [ ]:
for x, y in loader:
    print(x.shape, y.shape)

And we will use the vocabulary size to use as an output size for the model.
The model now takes:
- as input: a sequence of integers
- as output: for every possible BPE token, the probability that it is the next token in the sequence.

In [ ]:
# Define the vocab size based on the tokenizer
vocab_size = tokenizer.get_vocab_size()
vocab_size

We can now set up all the ingredients:
- the model uses 64 dimensions to represent the language
- we can calculate the loss (the difference between the actual next token and the predicted next token)
- the optimizer will tell the model in which direction to adjust the weights

In [21]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

from torch import nn, optim
# Hyperparameters
config = {
    "vocab_size": vocab_size,
    "embedding_dim": 64,
    "hidden_dim": 64,
    "num_layers": 2,
    "output_dim": vocab_size,
}

model = models.SlangRNN(config)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=50, min_lr=1e-4)

Let's train for 800 epochs. This means we will present the full dataset of 453 words for 800 times to the model.

In [ ]:
from loguru import logger
import torch

epochs = 800
history = []
last_lr = 0

for epoch in range(epochs):
    loss = 0

    for x, y in loader:
        optimizer.zero_grad()
        hidden = model.init_hidden(x)
        # input_seq, target_seq = dataset[i]

        output, hidden = model(x, hidden)

        loss += loss_fn(output.view(-1, vocab_size), y.view(-1))

    loss.backward()
    optimizer.step()
    scheduler.step(loss)
    history.append(loss.item())
    curr_lr = scheduler.get_last_lr()

    if (epoch+1) % 10 == 0:
        logger.info(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        if last_lr != curr_lr:
            last_lr = curr_lr
            logger.info(f"Current learning rate: {curr_lr}")


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history)
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.ylim(0, 40)

We can now save, and load, the trained model for use.

In [21]:
torch.save(model.state_dict(), "../artefacts/model.pth")

In [ ]:
model = models.SlangRNN(config)
model.load_state_dict(torch.load("../artefacts/model.pth"))

We can now give a starting letter, eg 'a', and give the model a sequence of start_token and start_letter.
The model will now start to predict next tokens, until it predicts the stop_token.

In [39]:
start_letter = 'a'
max_length = 20
temperature = 1.0
start_token_idx = tokenizer.encode("<s>").ids[0]
start_letter_idx = tokenizer.encode(start_letter).ids[0]
input_seq = torch.tensor([[start_token_idx, start_letter_idx]], dtype=torch.long)

generated_word = [start_letter_idx]
hidden = model.init_hidden(input_seq)
for _ in range(max_length - 1):
    with torch.no_grad():
        output, hidden = model(input_seq, hidden)
    output = output.squeeze(0)
    output = output[-1, :].view(-1).div(temperature).exp()
    next_token = torch.multinomial(output, 1).item()
    if next_token == tokenizer.token_to_id("<pad>"):
        break
    generated_word.append(next_token)
    input_seq = torch.tensor([generated_word], dtype=torch.long)

This generates tokens

In [ ]:
generated_word

Which we can decode into a word

In [ ]:
tokenizer.decode(generated_word)

We can loop this process to generate multiple words.

In [ ]:
models.sample_n(processed_words, n=10, model=model, tokenizer=tokenizer, max_length=20, temperature=1.0)

And save everything for later use.

In [29]:
tokenizer.save("../artefacts/tokenizer.json")

In [30]:
torch.save(model, "../artefacts/model.pth")